In [ ]:
!pip install thop

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_root = '/content/drive/MyDrive/ECE6953_Deep_Learning/Project'

import sys
import os
sys.path.append(os.path.join(drive_root, 'datasets'))

trained_model_path = os.path.join(drive_root, 'tmp_updated/HAM10000/0507_033609/test_best_HAM10000_0507_033609.pth')

Mounted at /content/drive


In [ ]:
import torch
import json
import torch.nn as nn  
import numpy as np
import matplotlib.pyplot as plt
# import torch.nn.functional as F
# import torch.optim as optim

# from torch.utils.tensorboard import SummaryWriter
# import prettytable
import time
sys.setrecursionlimit(15000)
# from thop.profile import profile

from PIL import Image
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchsummary import summary
from tqdm.notebook import tqdm
import random
# import seaborn as sns

# from utils import ImageShow,draw_size_acc,one_hot
# from utils import metrics_scores
from utils import pff, one_hot

from model import FixCapsNet

In [ ]:
def get_data(data_root, device, test_doc, T_size, num_workers, trans_test=312):
    data_transform = {
        "train": transforms.Compose([transforms.RandomResizedCrop((299, 299)),
                                     transforms.RandomVerticalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
        "val": transforms.Compose([transforms.Resize((302,302)),
                                   transforms.CenterCrop((299, 299)),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                  ]),
        "test": transforms.Compose([transforms.Resize((trans_test,trans_test)),
                                   transforms.CenterCrop((299, 299)),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                  ])
        }

    image_path = os.path.join(data_root,"datasets", "HAM100000")#
    print(image_path)
    assert os.path.exists(image_path), "{} path does not exist.".format(image_path)

    train_dataset = None
    test_dataset = None
    # train_dataset = datasets.ImageFolder(root=os.path.join(image_path,train_doc),#
    #                                      transform=data_transform["train"])
    # val_dataset = datasets.ImageFolder(root=os.path.join(image_path,val_doc),
    #                                         transform=data_transform["val"])
    test_dataset = datasets.ImageFolder(root=os.path.join(image_path,test_doc),
                                            transform=data_transform["test"])

    train_num = 0 # len(train_dataset)
    val_num = 0 # len(val_dataset)
    test_num = len(test_dataset)
    
    data_list = test_dataset.class_to_idx
    cla_dict = dict((val, key) for key, val in data_list.items())
    n_classes  = len(data_list)
    print(f'Using {n_classes } classes.')
    # write dict into json file
    # json_str = json.dumps(cla_dict, indent=4)
    # with open(f'{img_title}.json', 'w') as json_file:#class_indices
    #     json_file.write(json_str)
        
    pin_memory = True
    train_loader = None
    val_loader = None
    # train_loader = DataLoader(train_dataset,batch_size=BatchSize,
    #                                            pin_memory=pin_memory,
    #                                            shuffle=True,num_workers=num_workers)
    # val_loader = DataLoader(val_dataset,batch_size=V_size,
    #                                            pin_memory=pin_memory,
    #                                            shuffle=False,num_workers=num_workers)
    test_loader = DataLoader(test_dataset,batch_size=T_size,
                                              pin_memory=pin_memory,
                                              shuffle=False,num_workers=num_workers)

    print("using {} images for training, {} images for validation, {} images for testing.".format(train_num,
                                                                                                  val_num,
                                                                                                  test_num))
    return test_dataset, test_loader, test_num,n_classes,cla_dict

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
train_doc = "train525e384"
val_doc = "val525e384png"
test_doc = "test525png384"

BatchSize = 168
V_size = 40 
T_size = 24

nw = min([os.cpu_count(), BatchSize if BatchSize > 1 else 0, 6]) 
print(f'Using {nw} dataloader workers every process.')

test_dataset, test_loader, test_num, n_classes, cla_dict = get_data(drive_root, device, test_doc, T_size, nw, trans_test=313)

Using 2 dataloader workers every process.
/content/drive/MyDrive/ECE6953_Deep_Learning/Project/datasets/HAM100000
Using 7 classes.
using 0 images for training, 0 images for validation, 828 images for testing.


In [ ]:
# Create capsule network.
n_channels = 3
conv_outputs = 128 #Feature_map
num_primary_units = 8
primary_unit_size = 16 * 6 * 6  # fixme get from conv2d
output_unit_size = 16
img_size = 299
mode='128'
network = FixCapsNet(conv_inputs=n_channels,
                     conv_outputs=conv_outputs,
                     primary_units=num_primary_units,
                     primary_unit_size=primary_unit_size,
                     num_classes=n_classes,
                     output_unit_size=16,
                     init_weights=True,
                     mode=mode)
network = network.to(device)

dsize = (1, 3, 299, 299)
input_data = torch.randn(dsize).to(device)
pff(m_name="FixCaps"+'-'+mode,model=network,inputes=input_data)
summary(network,(n_channels,img_size,img_size))

+-------------+-----------+----------+-------+
|    Model    | Params(M) | FLOPs(G) |  FPS  |
+-------------+-----------+----------+-------+
| FixCaps-128 |    0.26   |   2.48   | 59.34 |
+-------------+-----------+----------+-------+
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 128, 141, 141]         124,544
              ReLU-2        [-1, 128, 141, 141]               0
FractionalMaxPool2d-3          [-1, 128, 20, 20]               0
            Conv2d-4          [-1, 128, 20, 20]          16,384
       BatchNorm2d-5          [-1, 128, 20, 20]             256
         Hardswish-6          [-1, 128, 20, 20]               0
 AdaptiveAvgPool2d-7            [-1, 128, 1, 1]               0
            Conv2d-8            [-1, 128, 1, 1]          16,384
              ReLU-9            [-1, 128, 1, 1]               0
           Conv2d-10            [-1, 128, 1, 1]          16

In [ ]:
network.load_state_dict(torch.load(trained_model_path)) #, strict=False)

<All keys matched successfully>

In [ ]:
def test(network, data_loader, n_classes, batch_size, delta=None):
    network.eval()
    correct_images = []
    pred_labels = []
    total_acc = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(tqdm(data_loader)):
            # batch_idx +=1
            # print(target)
            # target_indices = target#torch.Size([batch, 7])  
            data = data.to(device)
            if delta is not None:
                # Add perturbation if passed
                data += torch.tile(delta, (data.shape[0], 1, 1, 1))

            output = network(data).squeeze(dim=3)#torch.Size([batch_size, 7, 16, 1])  
            # print(output)       
            v_mag = torch.sqrt(torch.mean(output**2, dim=2, keepdim=False))
            pred = torch.argmax(v_mag, dim=1, keepdim=False).cpu() #[9, 2, 1, 1, 6,..., 1, 4, 6, 5, 7,]
            matched = (pred == target)
            # print(v_mag)
            # print(pred)
            # print(target)
            # print(matched)
            for i in range(target.shape[0]):
                pred_labels.append(pred[i].item())
                if matched[i].item():
                    correct_images.append(batch_idx*batch_size + i)
            total_acc += matched.type(torch.float32).mean().item()
            # break
    total_acc /= len(data_loader)
    print(f'Test accuracy: {total_acc*100.0:4.1f}')
    return correct_images, pred_labels

In [ ]:
def make_batches(dataset, batch_size, device):
    # Dataset must be [(tensor, label)*]
    tensors = []
    labels = []
    count = 0
    batches = []
    for t, l in dataset:
        # print(t.shape, l)
        tensors.append(t)
        labels.append(l)
        count += 1
        # break

        if count == batch_size:
            batches.append((torch.stack(tensors).to(device), torch.tensor(labels)))
            tensors = []
            labels = []
            count = 0
    if len(tensors) > 0:
        batches.append((torch.stack(tensors).to(device), torch.tensor(labels)))
    return batches

def save_np_img(nparr, filename):
    narr = np.transpose(nparr, (1, 2, 0))
    narr = ((narr + 1) / 2) * 255
    narr = narr.astype(np.uint8)
    im = Image.fromarray(narr)
    im.save(filename)

def classify(network, sample, device):
    with torch.no_grad():
        output = network(sample.to(device))
        v_mag = torch.sqrt(torch.mean(output**2, dim=2, keepdim=False))
        pred = torch.argmax(v_mag, dim=1, keepdim=False).cpu()

        # print(pred.flatten())
    return pred.flatten().numpy()

## UAP attack implementation

In [ ]:
def uap_attack(network, delta, data_loader, device, n_classes, epsilon=0.1):
    # Sample 1 random batch from the dataset
    batch_idx = random.randint(0, len(data_loader) - 1)
    data, target = data_loader[batch_idx]

    # Apply same delta to all images in the batch
    delta_splat = torch.tile(delta, (data.shape[0], 1, 1, 1))
    # We need gradients for this delta
    delta_splat.requires_grad = True

    # Create one hot vector for loss function
    target_one_hot = one_hot(target, length=n_classes).to(device)
    # Run the model
    output = network(data+delta_splat)
    # Compute loss and gradients
    loss = network.loss(output, target_one_hot, size_average=True)  
    loss.backward()

    # Compute this iteration's delta step using FGSM
    sgrad = epsilon * delta_splat.grad.detach().sign()
    
    # Accumulate the delta, averaging the step along the batch dimension
    delta += sgrad.mean(dim=0)

    return delta

In [ ]:
# First test to get correctly predicted images
correct_img_idx = test(network, test_loader, n_classes, T_size)
correct_dataset = [test_dataset[i] for i in correct_img_idx]

  0%|          | 0/35 [00:00<?, ?it/s]

Test accuracy: 93.8


In [ ]:
# UAP adversarial attack

# Initialize delta to 0
delta = torch.zeros(correct_dataset[0][0].shape).to(device)

accuracy = len(correct_dataset) / len(test_dataset)
# Accuracy threshold is 50%
while accuracy > 0.5:
    correct_loader = make_batches(correct_dataset, T_size, device)

    # Run the UAP attack and get an updated delta
    delta = uap_attack(network, delta, correct_loader, device, n_classes, epsilon=0.007)

    # Run the test with updated delta to get new list of correctly predicted images
    correct_img_idx = test(network, correct_loader, n_classes, T_size, delta)
    correct_dataset = [correct_dataset[i] for i in correct_img_idx]

    # Compute new accuracy
    accuracy = len(correct_dataset) / len(test_dataset)
    print(accuracy)

0.9359903381642513


  0%|          | 0/33 [00:00<?, ?it/s]

Test accuracy: 95.3
0.893719806763285


  0%|          | 0/31 [00:00<?, ?it/s]

Test accuracy: 95.8
0.856280193236715


...

Test accuracy: 93.0
0.5929951690821256


  0%|          | 0/21 [00:00<?, ?it/s]

Test accuracy: 90.7
0.5362318840579711


  0%|          | 0/19 [00:00<?, ?it/s]

Test accuracy: 89.7
0.47946859903381644


In [ ]:
delta_npy = delta.cpu().numpy()
max_delta, min_delta = np.max(delta_npy), np.min(delta_npy)
print("Range delta:", max_delta, min_delta)
# Save the delta
np.save(os.path.join(drive_root, "UAP_delta.npy"), delta_npy)
save_np_img(delta_npy, 'UAP_delta.png')

Range delta: 0.037916668 -0.040250003


In [ ]:
# Correct dataset indexes for examples
idxs = [4, 17, 47, 97, 100, 600, 770]
lbls = [correct_dataset[i][1] for i in idxs]
imgs = [correct_dataset[i][0].detach().cpu().numpy() for i in idxs]

# Save the original and perturbed examples
for i, n in enumerate(imgs):
    save_np_img(n, f"orig_{cla_dict[i]}.png")
    save_np_img(n + delta_npy, f"perturbed_{cla_dict[i]}.png")

In [ ]:
# Final test accuracy with UAP
_, pred_labels = test(network, test_loader, n_classes, T_size, delta=delta)

  0%|          | 0/35 [00:00<?, ?it/s]

Test accuracy: 35.1


## Boundary attack implementation

In [ ]:
test_loader_data = list(test_loader)

## Initial random init
sample, orig_label = test_loader_data[2]
adv_sample, adv_label = test_loader_data[33]

orig_label = orig_label.numpy()
adv_label = adv_label.numpy()
print(orig_label, adv_label)

pred_orig = classify(network, sample, device)
pred_adv = classify(network, adv_sample, device)

print(pred_orig, pred_adv)
orig_correct_mask = pred_orig == orig_label

[1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2] [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]


In [ ]:
def boundary_attack(orig, adv, delta=0.01, epsilon=0.001):
    full_shape = orig.shape
    orig = torch.reshape(orig, shape=(orig.shape[0], -1))
    adv = torch.reshape(adv, shape=(orig.shape[0], -1))

    unnorm_orig_adv = orig - adv
    d_orig_adv = torch.sqrt(torch.sum(unnorm_orig_adv**2, dim=1, keepdims=True))
    # print(d_orig_adv.flatten())
    norm_orig_adv = unnorm_orig_adv / d_orig_adv
    # select n_ki
    n_ki = torch.tensor(np.random.normal(size=norm_orig_adv.shape).astype(np.float32))

    # Orthogonal projection
    n_ki = n_ki - torch.sum(n_ki * norm_orig_adv, dim=1, keepdims=True)

    # Normalize
    d_n_ki = torch.sqrt(torch.sum((n_ki)**2, dim=1, keepdims=True))
    n_ki = delta * d_orig_adv * (n_ki / d_n_ki)

    # Spherical sample
    candidate = adv + n_ki

    # Move towards orig
    unnorm_orig_cand = orig - candidate
    d_orig_cand = torch.sqrt(torch.sum(unnorm_orig_cand**2, dim=1, keepdims=True))
    norm_orig_cand = unnorm_orig_cand / d_orig_cand
    # print(candidate.shape, norm_orig_cand.shape, d_orig_adv.shape)
    candidate = candidate + epsilon * norm_orig_cand * d_orig_adv

    unnorm_orig_cand = orig - candidate
    d_orig_cand = torch.sqrt(torch.sum(unnorm_orig_cand**2, dim=1, keepdims=True))

    # For checking
    d_n_ki = torch.sqrt(torch.sum((adv-candidate)**2, dim=1, keepdims=True))
    return torch.reshape(candidate, shape=full_shape)
    # print(d_n_ki/d_orig_adv)
    # print(1-d_orig_cand/d_orig_adv)
# print(n_ki.shape)

In [ ]:
for i in range(100):
    candidate = boundary_attack(sample, adv_sample, delta=0.01, epsilon=0.01)
    pred_adv = classify(network, candidate, device)
    # Check which among candidates are still adversarial
    # Mask out the elems which were initially wrong
    incorrect_mask = np.logical_and(pred_adv != orig_label, orig_correct_mask)
    incorrect_sel = torch.reshape(torch.tensor(incorrect_mask.astype(np.float32)), shape=(sample.shape[0], 1, 1, 1))
    adv_sample = (1-incorrect_sel)*adv_sample + incorrect_sel * candidate

    unmatched = np.sum(incorrect_mask)
    print("Updates", unmatched, pred_adv)

Updates 16 [5 5 5 5 5 2 5 5 2 2 2 5 5 2 5 5 2 5 5 5 5 5 5 2]
Updates 18 [5 5 5 5 5 5 5 5 2 2 2 5 5 5 5 5 2 5 5 5 5 5 5 2]
Updates 17 [5 5 5 5 5 5 5 5 2 2 2 5 5 2 5 5 2 5 5 5 5 5 5 2]
Updates 18 [5 5 5 5 5 5 5 5 2 2 2 5 5 5 5 5 2 5 5 5 5 5 5 2]
Updates 16 [5 5 5 5 5 2 5 5 2 2 2 5 5 2 5 5 2 5 5 5 5 5 5 2]
Updates 18 [5 5 5 5 5 5 5 5 2 2 2 5 5 5 5 5 2 5 5 5 5 5 5 2]
Updates 17 [5 5 5 5 5 5 5 5 2 2 2 5 5 2 5 5 2 5 5 5 5 5 5 2]
Updates 17 [5 5 5 5 5 5 5 5 2 2 2 5 5 2 5 5 2 5 5 5 5 5 5 2]
Updates 17 [5 5 5 5 5 5 5 5 2 2 2 5 5 2 5 5 2 5 5 5 5 5 5 2]
Updates 17 [5 5 5 5 5 5 5 5 2 2 2 5 5 2 5 5 2 5 5 5 5 5 5 2]
Updates 18 [5 5 5 5 5 5 5 5 2 2 2 5 5 2 5 5 5 5 5 5 5 5 5 2]
Updates 17 [5 5 5 5 5 5 5 5 2 2 2 5 5 2 5 5 2 5 5 5 5 5 5 2]
Updates 18 [5 5 5 5 5 5 5 5 2 2 2 5 5 2 5 5 5 5 5 5 5 5 5 2]
Updates 20 [5 5 5 5 5 5 5 5 2 5 2 5 5 5 5 5 5 5 5 5 5 5 5 2]
Updates 18 [5 5 5 5 5 5 5 5 2 2 2 5 5 2 5 5 5 5 5 5 5 5 5 2]
Updates 17 [5 5 5 5 5 5 5 5 2 2 2 5 5 2 5 5 2 5 5 5 5 5 5 2]
Updates 17 [5 5 5 5 5 5 

KeyboardInterrupt: ignored

In [ ]:
pred_adv = classify(network, adv_sample, device)
print(pred_adv)
print(orig_label)

[5 2 2 2 2 2 2 2 2 2 2 2 5 2 5 2 2 2 2 2 5 5 2 2]
[1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
